# Step 3: Feature extraction (image → vector)

Implement in src/features.py later
### Techniques:
- Color histogram → simple RGB histogram vector
- Texture features → Local Binary Patterns (LBP)
- Shape/structure features → Histogram of Oriented Gradients (HOG)
### Output: 1D fixed-length feature vector for each image.
- Save extracted features (e.g., numpy arrays) to avoid re-computation.
### Start with HOG + color histogram for a strong baseline.

> Raw images are 2D (height × width × channels).
SVM and kNN require 1D vectors per sample.
Features should capture color, shape, and texture of trash items, which helps the classifier distinguish metal, paper, plastic, etc.

In [ ]:
import cv2
import numpy as np
from skimage.feature import hog
from skimage.color import rgb2gray

In [ ]:
# Color histogram
def color_histogram(img, bins=32):
    # img: RGB image
    hist_r = cv2.calcHist([img], [0], None, [bins], [0, 256])
    hist_g = cv2.calcHist([img], [1], None, [bins], [0, 256])
    hist_b = cv2.calcHist([img], [2], None, [bins], [0, 256])
    hist = np.concatenate([hist_r, hist_g, hist_b]).flatten()
    hist = hist / np.sum(hist)  # normalize
    return hist

# HOG features
def hog_features(img, pixels_per_cell=(16,16), cells_per_block=(2,2)):
    gray = rgb2gray(img)
    features = hog(gray,
                   pixels_per_cell=pixels_per_cell,
                   cells_per_block=cells_per_block,
                   block_norm='L2-Hys')
    return features

# Combine HOG + color histogram
def combined_features(img):
    hist = color_histogram(img)
    hog_feat = hog_features(img)
    return np.concatenate([hist, hog_feat])
